# Data from stt.pecha.tools

Connect to the postgres databse hosted on Render using dbeaver-ce and run the following query and export the result as a csv file. 

![export_csv](data/export_csv.png)


### All transcribed data
Use the following query when you want more data but willing to compromise one the quality of the transcription. Task in state 'submitted' means the task has been transcribed by the first transcriber and not yet been reviewed or done quality control on.
```sql
select t.transcript, t.reviewed_transcript, t.final_transcript, t.file_name, t.url 
from "Task" t 
where t.state in ('submitted', 'accepted', 'finalised');
```

### Reviewed only
The following query selects only the task that has been reviewed by the team lead but may or may not have been reviewed by the quality control team.
reviewed at least once
```sql 
select t.reviewed_transcript, t.final_transcript, t.file_name, t.url 
from "Task" t 
where t.state in ('accepted', 'finalised');
```

Read the exported csv from stt.pecha.tools

In [13]:
import pandas as pd

csv_path = "/home/monlamai/_Task__202401171255.csv"
df = pd.read_csv(csv_path)

In [14]:
df.head()

,reviewed_transcript,final_transcript,file_name,url
0,ཉི་མ་རིང་ཐུང་ལའང་མ་འདོད་པ་བྱས་ཀྱི་ཡོད་ན་ངས་ཚེ་...,NaN,STT_AB00237_0190_1711732_to_1723319,https://huggingface.co/datasets/openpecha/STT_...
1,ད་ཆ་ཁྱིམ་ཚང་འདི་ལ་འགྲིག་མཐུན་གྱི་ཆ་རྐྱེན་གནས་ཡ...,NaN,STT_AB00237_0197_1767500_to_1776441,https://huggingface.co/datasets/openpecha/STT_...
2,དྒྱེ་ནས་ཕ་དོན་གྲུབ་ཀྱི་བསམ་བློའི་ནང་དེ་ནི་ཕལ་ཆ...,NaN,STT_AB00238_0005_30554_to_39889,https://huggingface.co/datasets/openpecha/STT_...
3,དེ་ཉིན་གང་ལྟར་དོན་གྲུབ་ཁྱིམ་དུ་སླེབས་མ་ཐག་རང་ག...,NaN,STT_AB00238_0006_41271_to_46288,https://huggingface.co/datasets/openpecha/STT_...
4,དོན་གྲུབ་རང་སྐྱེས་པ་ཁྱོ་ག་ཞིག་ཡིན་པས་,NaN,STT_AB00238_0007_47056_to_49957,https://huggingface.co/datasets/openpecha/STT_...


grade column is used to indicate if the quality of the transcription 
|grade|meaning|
|-----|--------|
| 1   | Transcribed only|
| 2   | Reviewed by the team lead|
| 3   | Reviewed by the quality control team|

In [15]:
df['grade'] = 1 # by default all the task are transcibed

In [16]:
df.loc[~df['reviewed_transcript'].isna(), 'grade'] = 2 # if the task has been reviewed, grade = 2

In [17]:
df.loc[~df['final_transcript'].isna(), 'grade'] = 3 # if the task has been finalized, grade = 3

In [18]:
df['final_transcript'].fillna(df['reviewed_transcript'], inplace=True)

In [19]:
df.head()

,reviewed_transcript,final_transcript,file_name,url,grade
0,ཉི་མ་རིང་ཐུང་ལའང་མ་འདོད་པ་བྱས་ཀྱི་ཡོད་ན་ངས་ཚེ་...,ཉི་མ་རིང་ཐུང་ལའང་མ་འདོད་པ་བྱས་ཀྱི་ཡོད་ན་ངས་ཚེ་...,STT_AB00237_0190_1711732_to_1723319,https://huggingface.co/datasets/openpecha/STT_...,2
1,ད་ཆ་ཁྱིམ་ཚང་འདི་ལ་འགྲིག་མཐུན་གྱི་ཆ་རྐྱེན་གནས་ཡ...,ད་ཆ་ཁྱིམ་ཚང་འདི་ལ་འགྲིག་མཐུན་གྱི་ཆ་རྐྱེན་གནས་ཡ...,STT_AB00237_0197_1767500_to_1776441,https://huggingface.co/datasets/openpecha/STT_...,2
2,དྒྱེ་ནས་ཕ་དོན་གྲུབ་ཀྱི་བསམ་བློའི་ནང་དེ་ནི་ཕལ་ཆ...,དྒྱེ་ནས་ཕ་དོན་གྲུབ་ཀྱི་བསམ་བློའི་ནང་དེ་ནི་ཕལ་ཆ...,STT_AB00238_0005_30554_to_39889,https://huggingface.co/datasets/openpecha/STT_...,2
3,དེ་ཉིན་གང་ལྟར་དོན་གྲུབ་ཁྱིམ་དུ་སླེབས་མ་ཐག་རང་ག...,དེ་ཉིན་གང་ལྟར་དོན་གྲུབ་ཁྱིམ་དུ་སླེབས་མ་ཐག་རང་ག...,STT_AB00238_0006_41271_to_46288,https://huggingface.co/datasets/openpecha/STT_...,2
4,དོན་གྲུབ་རང་སྐྱེས་པ་ཁྱོ་ག་ཞིག་ཡིན་པས་,དོན་གྲུབ་རང་སྐྱེས་པ་ཁྱོ་ག་ཞིག་ཡིན་པས་,STT_AB00238_0007_47056_to_49957,https://huggingface.co/datasets/openpecha/STT_...,2


In [20]:
df = df[~df['final_transcript'].isnull()]

In [21]:
df['dept'] = df['file_name'].str[:6] # extract department code from file name

In [22]:
df = df[['file_name','final_transcript', 'url', 'dept', 'grade']]

In [23]:
df.rename(columns={'final_transcript':'uni'}, inplace=True)

In [24]:
df.head()

,file_name,uni,url,dept,grade
0,STT_AB00237_0190_1711732_to_1723319,ཉི་མ་རིང་ཐུང་ལའང་མ་འདོད་པ་བྱས་ཀྱི་ཡོད་ན་ངས་ཚེ་...,https://huggingface.co/datasets/openpecha/STT_...,STT_AB,2
1,STT_AB00237_0197_1767500_to_1776441,ད་ཆ་ཁྱིམ་ཚང་འདི་ལ་འགྲིག་མཐུན་གྱི་ཆ་རྐྱེན་གནས་ཡ...,https://huggingface.co/datasets/openpecha/STT_...,STT_AB,2
2,STT_AB00238_0005_30554_to_39889,དྒྱེ་ནས་ཕ་དོན་གྲུབ་ཀྱི་བསམ་བློའི་ནང་དེ་ནི་ཕལ་ཆ...,https://huggingface.co/datasets/openpecha/STT_...,STT_AB,2
3,STT_AB00238_0006_41271_to_46288,དེ་ཉིན་གང་ལྟར་དོན་གྲུབ་ཁྱིམ་དུ་སླེབས་མ་ཐག་རང་ག...,https://huggingface.co/datasets/openpecha/STT_...,STT_AB,2
4,STT_AB00238_0007_47056_to_49957,དོན་གྲུབ་རང་སྐྱེས་པ་ཁྱོ་ག་ཞིག་ཡིན་པས་,https://huggingface.co/datasets/openpecha/STT_...,STT_AB,2


In [26]:
df['file_name'] = df['file_name'].map(lambda x: x.replace(".wav", ""))
df['file_name'] = df['file_name'].map(lambda x: x.replace(".mp3", ""))

In [29]:
df.to_csv('01_stt_pecha_tools.tsv', sep='\t', index=False)

In [30]:
df.shape

(168277, 5)